In [1]:
import pyspark
from pyspark.sql import SparkSession
import warnings
warnings.filterwarnings('ignore')
import warnings; warnings.simplefilter('ignore')
import pandas as pd
from pyspark.sql.functions import *
from google.cloud import storage
from pyspark.sql import types
from pyspark.sql.functions import date_format
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [3]:
spark = SparkSession.builder \
        .master("local") \
        .appName("Boston Service Request") \
        .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

In [4]:
schema = types.StructType([
types.StructField('case_enquiry_id', types.StringType(),True),
types.StructField('open_dt', types.StringType(),True),
types.StructField('target_dt', types.TimestampType(),True),
types.StructField('closed_dt', types.TimestampType(),True),
types.StructField('ontime', types.StringType(),True),
types.StructField('case_status', types.StringType(),True),
types.StructField('closure_reason', types.StringType(),True),
types.StructField('case_title', types.StringType(),True),
types.StructField('subject', types.StringType(),True),
types.StructField('reason', types.StringType(),True),
types.StructField('type', types.StringType(),True),
types.StructField('queue', types.StringType(),True),
types.StructField('department', types.StringType(),True),
types.StructField('submittedphoto', types.StringType(),True),
types.StructField('closedphoto', types.StringType(),True),
types.StructField('location', types.StringType(),True),
types.StructField('fire_district', types.StringType(),True),
types.StructField('pwd_district', types.StringType(),True),
types.StructField('city_council_district', types.StringType(),True),
types.StructField('police_district', types.StringType(),True),
types.StructField('neighborhood', types.StringType(),True),
types.StructField('neighborhood_services_district', types.StringType(),True),
types.StructField('ward', types.StringType(),True),
types.StructField('precinct', types.StringType(),True),
types.StructField('location_street_name', types.StringType(),True),
types.StructField('location_zipcode', types.StringType(),True),
types.StructField('latitude', types.StringType(),True),
types.StructField('longitude', types.StringType(),True),
types.StructField('source', types.StringType(),True)
])

In [102]:
pandasDF = df.toPandas()
spark.createDataFrame(pandasDF).schema

KeyboardInterrupt: 

In [5]:
df = spark.read.option("header",True).schema(schema).csv("gs://datalake-311-bronze/boston_2022.csv")



In [6]:

df.dtypes
df.printSchema()

root
 |-- case_enquiry_id: string (nullable = true)
 |-- open_dt: string (nullable = true)
 |-- target_dt: timestamp (nullable = true)
 |-- closed_dt: timestamp (nullable = true)
 |-- ontime: string (nullable = true)
 |-- case_status: string (nullable = true)
 |-- closure_reason: string (nullable = true)
 |-- case_title: string (nullable = true)
 |-- subject: string (nullable = true)
 |-- reason: string (nullable = true)
 |-- type: string (nullable = true)
 |-- queue: string (nullable = true)
 |-- department: string (nullable = true)
 |-- submittedphoto: string (nullable = true)
 |-- closedphoto: string (nullable = true)
 |-- location: string (nullable = true)
 |-- fire_district: string (nullable = true)
 |-- pwd_district: string (nullable = true)
 |-- city_council_district: string (nullable = true)
 |-- police_district: string (nullable = true)
 |-- neighborhood: string (nullable = true)
 |-- neighborhood_services_district: string (nullable = true)
 |-- ward: string (nullable = true)


In [ ]:
df = df.repartition(10)

In [85]:
df.write.format('bigquery') \
    .option('project','dataengineering-bizzy') \
    .option('table','boston_service_request.boston_311_raw') \
    .option("temporaryGcsBucket","datalake-311-silver") \
    .mode("overwrite") \
    .save()

In [ ]:
df.write.format("parquet").option("path", "gs://datalake-311-silver/archive/").save()

In [88]:
df = spark.read.format('bigquery') \
    .option('project','dataengineering-bizzy') \
    .option('table','boston_service_request.boston_311_raw') \
    .load()

In [89]:
df.describe(["case_enquiry_id"]).show() #max returning a value and not ? means no nulls in this column

+-------+--------------------+
|summary|     case_enquiry_id|
+-------+--------------------+
|  count|              273951|
|   mean|1.010038463896175...|
| stddev|   153628.0535129675|
|    min|        101003578872|
|    max|        101004113296|
+-------+--------------------+



In [106]:
for col in df.columns:
    df.describe([col]).show()

+-------+--------------------+
|summary|     case_enquiry_id|
+-------+--------------------+
|  count|              273951|
|   mean|1.010038463895628...|
| stddev|  153628.05383988871|
|    min|        101003578872|
|    max|        101004113296|
+-------+--------------------+



+-------+-------------------+
|summary|            open_dt|
+-------+-------------------+
|  count|             273951|
|   mean|               null|
| stddev|               null|
|    min|2021-01-01 00:06:37|
|    max|2021-12-31 23:44:48|
+-------+-------------------+

+-------+
|summary|
+-------+
|  count|
|   mean|
| stddev|
|    min|
|    max|
+-------+

+-------+
|summary|
+-------+
|  count|
|   mean|
| stddev|
|    min|
|    max|
+-------+



+-------+-------+
|summary| ontime|
+-------+-------+
|  count| 273951|
|   mean|   null|
| stddev|   null|
|    min| ONTIME|
|    max|OVERDUE|
+-------+-------+



+-------+-----------+
|summary|case_status|
+-------+-----------+
|  count|     273951|
|   mean|       null|
| stddev|       null|
|    min|     Closed|
|    max|       Open|
+-------+-----------+



+-------+--------------------+
|summary|      closure_reason|
+-------+--------------------+
|  count|              273951|
|   mean|                null|
| stddev|                null|
|    min|                    |
|    max|Case Closed. Clos...|
+-------+--------------------+



...

+-------+--------------------+
|summary|location_street_name|
+-------+--------------------+
|  count|              271129|
|   mean|                null|
| stddev|                null|
|    min|  0 Saint Paul's Ave|
|    max|    Pier 4 Eighth St|
+-------+--------------------+



+-------+-----------------+
|summary| location_zipcode|
+-------+-----------------+
|  count|           212881|
|   mean| 2126.85175285723|
| stddev|17.31285726352793|
|    min|            02108|
|    max|            02467|
+-------+-----------------+



+-------+-------------------+
|summary|           latitude|
+-------+-------------------+
|  count|             273951|
|   mean|  42.33437729125488|
| stddev|0.03247340544116824|
|    min|            42.2302|
|    max|            42.3952|
+-------+-------------------+



+-------+-------------------+
|summary|          longitude|
+-------+-------------------+
|  count|             273951|
|   mean|   -71.076193295564|
| stddev|0.03287928803900102|
|    min|           -70.9949|
|    max|           -71.1818|
+-------+-------------------+



+-------+--------------------+
|summary|              source|
+-------+--------------------+
|  count|              273951|
|   mean|                null|
| stddev|                null|
|    min|Citizens Connect App|
|    max|        Self Service|
+-------+--------------------+



In [90]:
df.columns

['case_enquiry_id',
 'open_dt',
 'target_dt',
 'closed_dt',
 'ontime',
 'case_status',
 'closure_reason',
 'case_title',
 'subject',
 'reason',
 'type',
 'queue',
 'department',
 'location',
 'fire_district',
 'neighborhood',
 'source']

In [91]:
df =df.drop("latitude", "longitude", "submittedphoto","police_district", "location_zipcode", "ward", "closedphoto","police_district", "neighborhood_services_district","queue" "fire_district","city_council_district","precinct", "location_street_name", "pwd_district")

In [92]:
df.printSchema()

root
 |-- case_enquiry_id: string (nullable = true)
 |-- open_dt: string (nullable = true)
 |-- target_dt: timestamp (nullable = true)
 |-- closed_dt: timestamp (nullable = true)
 |-- ontime: string (nullable = true)
 |-- case_status: string (nullable = true)
 |-- closure_reason: string (nullable = true)
 |-- case_title: string (nullable = true)
 |-- subject: string (nullable = true)
 |-- reason: string (nullable = true)
 |-- type: string (nullable = true)
 |-- queue: string (nullable = true)
 |-- department: string (nullable = true)
 |-- location: string (nullable = true)
 |-- fire_district: string (nullable = true)
 |-- neighborhood: string (nullable = true)
 |-- source: string (nullable = true)



In [12]:
for col in df.columns:
    print(col, "\t", "with '?' values: ", df.filter(df[col]=="?").count())

case_enquiry_id 	 with '?' values:  0


open_dt 	 with '?' values:  0
target_dt 	 with '?' values:  0
closed_dt 	 with '?' values:  0


ontime 	 with '?' values:  0


case_status 	 with '?' values:  0


closure_reason 	 with '?' values:  0


...

department 	 with '?' values:  0


location 	 with '?' values:  0


fire_district 	 with '?' values:  0


neighborhood 	 with '?' values:  0


source 	 with '?' values:  0


In [93]:
df2 = df.replace('?',None)

In [ ]:
for col in df2.columns:
    print(col, "\t", "with '?' values: ", df2.filter(df2[col]=="?").count())

In [94]:
df_time = df2.withColumn('resolution_time_sec',round(unix_timestamp("closed_dt")) - round(unix_timestamp('open_dt')))\
             .withColumn('resolution_time_mins',round(unix_timestamp("closed_dt")/60) - round(unix_timestamp('open_dt')/60))

In [86]:
 df_time.write.format('bigquery') \
    .option('project','dataengineering-bizzy') \
    .option('table','boston_service_request.boston_service_summary') \
    .option("temporaryGcsBucket","datalake-311-silver") \
    .mode("overwrite") \
    .save()

In [95]:
df_time.show()

+---------------+-------------------+-------------------+-------------------+-------+-----------+--------------------+------------+--------------------+-------------------+--------------------+--------------------+----------+--------------------+-------------+------------+--------------------+-------------------+--------------------+
|case_enquiry_id|            open_dt|          target_dt|          closed_dt| ontime|case_status|      closure_reason|  case_title|             subject|             reason|                type|               queue|department|            location|fire_district|neighborhood|              source|resolution_time_sec|resolution_time_mins|
+---------------+-------------------+-------------------+-------------------+-------+-----------+--------------------+------------+--------------------+-------------------+--------------------+--------------------+----------+--------------------+-------------+------------+--------------------+-------------------+--------------

In [96]:
df2 =df_time.na.fill(value=0,subset=["resolution_time_sec", "resolution_time_mins"])

In [99]:
df2.groupBy("source").count().alias("total").show(10)

+--------------------+------+
|              source| count|
+--------------------+------+
|        Self Service|  8662|
|  Maximo Integration|     1|
|  Employee Generated|  7126|
|    Constituent Call|100674|
|Citizens Connect App|132992|
|     City Worker App| 24496|
+--------------------+------+



In [100]:
df2.createOrReplaceTempView("service_request")


### 1. Most used channel for request

In [101]:
spark.sql("select source, count(source) as Total from service_request group by source order by Total desc").show(truncate=False)

+--------------------+------+
|source              |Total |
+--------------------+------+
|Citizens Connect App|132992|
|Constituent Call    |100674|
|City Worker App     |24496 |
|Self Service        |8662  |
|Employee Generated  |7126  |
|Maximo Integration  |1     |
+--------------------+------+



### 2. Percent of ticket resolved ontime

In [103]:
df2.groupBy("ontime").count().alias("total").show(10)

spark.sql("select ontime, count(ontime), \
          SUM(COUNT(ontime)) OVER() AS total_count, \
          count(ontime) *100.0 /sum(count(ontime)) over () as request_percent \
          from service_request group by ontime").show(truncate=False)

+-------+------+
| ontime| count|
+-------+------+
|OVERDUE| 45366|
| ONTIME|228585|
+-------+------+



+-------+-------------+-----------+-----------------+
|ontime |count(ontime)|total_count|request_percent  |
+-------+-------------+-----------+-----------------+
|OVERDUE|45366        |273951     |16.55989574777971|
|ONTIME |228585       |273951     |83.44010425222029|
+-------+-------------+-----------+-----------------+



### 3. Maximum no of days to resolve a ticket in each category

In [104]:
spark.sql("select case_enquiry_id,open_dt,closed_dt,case_title, resolution_time_mins, (resolution_time_mins/60/24) as resolution_days from "\
          "(select * ,row_number() OVER (PARTITION BY type ORDER BY resolution_time_mins DESC) as rn "\
          " from service_request) tmp where rn <=1").show(truncate=False)

+---------------+-------------------+-------------------+----------------------------------+--------------------+--------------------+
|case_enquiry_id|open_dt            |closed_dt          |case_title                        |resolution_time_mins|resolution_days     |
+---------------+-------------------+-------------------+----------------------------------+--------------------+--------------------+
|101003580807   |2021-01-04 11:46:00|2021-09-20 10:17:03|Abandoned Bicycle                 |372811.0            |258.89652777777775  |
|101003647496   |2021-02-23 20:37:00|2021-12-07 10:21:18|Abandoned Building                |412664.0            |286.57222222222225  |
|101003684008   |2021-03-22 08:38:00|2021-12-07 10:37:23|Abandoned Vehicles                |374579.0            |260.12430555555557  |
|101003714274   |2021-04-17 17:06:00|null               |Aircraft Noise Disturbance        |0.0                 |0.0                 |
|101003653200   |2021-03-01 15:03:31|2021-03-01 15:05:0

### 4. Most frequent request

In [105]:
spark.sql("select case_title, count(case_title) as total from service_request group by case_title order by total desc").show(truncate=False)

+-----------------------------------+-----+
|case_title                         |total|
+-----------------------------------+-----+
|Parking Enforcement                |57414|
|Requests for Street Cleaning       |18570|
|Improper Storage of Trash (Barrels)|13587|
|Schedule a Bulk Item Pickup        |11841|
|CE Collection                      |11801|
|Needle Pickup                      |7864 |
|Request for Pothole Repair         |7362 |
|Poor Conditions of Property        |5972 |
|Tree Maintenance Requests          |5637 |
|Sign Repair                        |5297 |
|Pick up Dead Animal                |5289 |
|Abandoned Vehicles                 |5088 |
|Schedule Bulk Item Pickup          |5016 |
|Graffiti Removal                   |4902 |
|Request for Recycling Cart         |4663 |
|Sidewalk Repair (Make Safe)        |4586 |
|Traffic Signal Inspection          |4146 |
|Recycling Cart Return              |3988 |
|Rodent Activity                    |3586 |
|Animal Generic Request         

### 5. Average Resolution Time of a Case Type

In [120]:
spark.sql("select case_title, average_res_time_mins, (average_res_time_mins/60/24) as resolution_days from "\
          "(select * ,AVG(resolution_time_mins) OVER (PARTITION BY case_title ORDER BY case_title DESC) as average_res_time_mins "\
          "from service_request) group by case_title, average_res_time_mins,(average_res_time_mins/60/24) \
          order by case when case_title is null then 1 else 0 end, case_title ").show(truncate=False)




+------------------------------------------------------------------+---------------------+--------------------+
|case_title                                                        |average_res_time_mins|resolution_days     |
+------------------------------------------------------------------+---------------------+--------------------+
| Maintenance Complaint - Residential                              |318.0                |0.22083333333333333 |
| Mice Infestation - Residential                                   |8.0                  |0.005555555555555556|
| No Utilities Residential - Electricity                           |1399.0               |0.9715277777777778  |
| Pest Infestation - Residential                                   |2058.5               |1.4295138888888888  |
| Rodent Activity                                                  |8567.0               |5.949305555555555   |
| SCHEDULED Chronic Dampness/Mold                                  |354257.0             |246.0118055555